Notebook to predict N+1 command where N is 1..4 and command sets can be up to len 5

Construct Graph Function. Also contains `adjust_session` which removes duplicate commands in subsets.

In [7]:
from graph_no_fuzz import Node

def adjust_session(session, frequencies = []):
    if len(session) == 2:
        if session[0] == session[1]:
            session = [session[0]]
            frequencies = [2]
        else:
            frequencies = [1] * len(session)
    elif len(session) == 3:
        if session[0] == session[1] and session[0] == session[2]:
            session = session[:1]
            frequencies = [3]
        else:
            frequencies = [1] * len(session)
    elif len(session) == 4:
        if session[0] == session[2] and session[1] == session[3]:
            session = [session[0], session[1]]
            frequencies = [2, 2]
        else:
            frequencies = [1] * len(session)
    elif len(session) == 5:
        if session[0] == session[1] and session[0] == session[2]:
            session = [session[0], session[3], session[4]]
            frequencies = [3, 1, 1]
        elif session[0] == session[2] and session[1] == session[3]:
            session = [session[0], session[1], session[4]]
            frequencies = [2, 2, 1]
        else:
            frequencies = [1] * len(session)
    else:
        frequencies = [1] * len(session)

    return session, frequencies


def construct_graph(command_list, command_dict={}):

    filter_empty = lambda x: (len(x) > 0)
    cur_node = None
    child_node = None
    
    for session in command_list:
        # remove commands of length 0 ( not good to modify list within loop )
        session = list(filter(filter_empty, session))

        try: 
            first_cmd = session[0]
        except Exception as inst:
            print(session)
            continue

        program = session[0].split()[0]

        # change this to frequency array of length program
        # and add frequency based on index to it    
        freq_addition = []
        session, freq_addition = adjust_session(session=session)

        if command_dict.get(program) is None:
            cur_node = Node(program=program, frequency=freq_addition[0])
            command_dict[program] = cur_node 
        else:
            cur_node = command_dict.get(program)
            cur_node.frequency += freq_addition[0]

        # what does this do?
        if cur_node.commands.get(first_cmd) is None:
            cur_node.commands[first_cmd] = freq_addition[0]
        else:
            cur_node.commands[first_cmd] += freq_addition[0]

        for cmd in range(1, len(session)):

            # remove
            if cmd >= len(freq_addition):
                    print(session, len(session))
                    print(freq_addition, len(freq_addition))

            if session[cmd].isprintable() is False:
                break
            else:
                program = session[cmd].split()[0]

            if cur_node.children.get(program) is None:
                child_node = Node(program = program, frequency=freq_addition[cmd])
                cur_node.children[program] = child_node
            else:
                child_node = cur_node.children.get(program)
                child_node.frequency += freq_addition[cmd]
            
            # ?????
            if child_node.commands.get(session[cmd]) is None:
                child_node.commands[session[cmd]] = freq_addition[cmd]
            else:
                child_node.commands[session[cmd]] += freq_addition[cmd]
            
            cur_node = child_node
        
    return command_dict

Get Prediction Function. Traverses down graph and returns prediction if there is one, else returns None

In [8]:
def get_prediction(command_list: list[str], graph, result_size=5, graph_depth=4):
    if len(command_list) == 0:
        return None

    commands = command_list[:graph_depth]
    # should be labeled last command
    last_command = command_list[graph_depth]

    if len(commands) == 0:
        return None, [-1]

    program = commands[0].split()[0]

    frequencies = []
    
    if graph.get(program) is None:
        frequencies.append(-1)
        return None, frequencies
    else:
        node = graph[program]
        frequencies.append(node.frequency)

    for command in commands[1:]:
        program = command.split()[0]
        if node.children.get(program) is not None:
            node = node.children[program]
            frequencies.append(node.frequency)
        else:
            frequencies.append(-1)
            return None, frequencies
    
    # last frequency if it has it
    next_program = last_command.split()[0]

    if node.children.get(next_program) is not None:
        frequencies.append(node.children[next_program].frequency)
    else:
        frequencies.append(-1)
    
    # previous command should be command_list[depth-1]
    return node.get_prediction(previous_command=command_list[graph_depth-1], num_to_return=result_size), frequencies

Accuracy Function. Uses fuzzy matching to determine accuracy up to a threshold. Boolean parameters in `get_accuracy` determine logging successes and fails to lists that can be printed later. `get_accuracy` checks if the correct prediction was in the first 3, 5, and 15 predictions. 

In [9]:
from thefuzz import fuzz
from thefuzz import process

from statistics import mean

from tqdm import tqdm

from sklearn.model_selection import train_test_split

import pprint
pp = pprint.PrettyPrinter(indent=4)

def append_list(lst, results, commands, graph_depth):
    lst.append({"Results": results, "Expected": commands[graph_depth], "Command Sequence": commands})

def get_accuracy(command_subsets, fail_list = [], fail_frequencies= [], succeed_list = [], succeed_frequencies = [], correct_5_not_3= [], correct_15_not_5 = [], print_fails=False, graph_depth=-1):
    train_data, test_data = train_test_split(command_subsets, test_size=0.2, train_size=0.8, random_state=42)

    train_graph = construct_graph(train_data)

    # filter out nodes that can't get prediction because not enough length so don't contribute and will cause error
    if (graph_depth != -1):
        test_data = [x for x in test_data if len(x) > graph_depth]
    else:
        test_data = [x for x in test_data if len(x) >= 2]
        
    test_size = len(test_data)

    return_3 = 3
    return_5 = 5
    return_15 = 15

    correct = 0
    correct_with_5 = 0
    correct_with_15 = 0
    first_prediction = 0
    has_prediction = 0
    incorrect = 0
    none_count = 0

    for commands in tqdm(test_data, leave=False):
        results_3, frequencies = get_prediction(commands, train_graph, return_3, graph_depth)
        results_5, _ = get_prediction(commands, train_graph, return_5, graph_depth)
        results_15, _ = get_prediction(commands, train_graph, return_15, graph_depth)

        command_freq = [(commands[i], frequencies[i]) for i in range(len(frequencies))]
        
        prev_correct = correct
        prev5_correct =  correct_with_5

        if results_3 is not None:
            has_prediction += 1

            for i in range(len(results_3)):
                if fuzz.ratio(results_3[i][0], commands[graph_depth]) > 85:
                    correct += 1

                    succeed_frequencies.append(mean(frequencies))

                    if correct <= 100:
                        append_list(succeed_list, results_5, command_freq, graph_depth)

                    if i == 0:
                        first_prediction += 1
                
                    break

            if prev_correct == correct:
                fail_frequencies.append(mean(frequencies))
                incorrect += 1

            for i in range(len(results_5)):
                if fuzz.ratio(results_5[i][0], commands[graph_depth]) > 85:
                    correct_with_5 += 1

                    if correct_with_5 - correct <= 100 and prev_correct == correct:
                        append_list(correct_5_not_3, results_5, command_freq, graph_depth)

                    break

            for i in range(len(results_15)):
                if fuzz.ratio(results_15[i][0], commands[graph_depth]) > 85:
                    correct_with_15 += 1

                    if correct_with_15 - correct <= 100 and prev5_correct == correct_with_5:
                        append_list(correct_15_not_5, results_15, command_freq, graph_depth)

                    break

        else:
            none_count += 1

        if results_5 is not None and prev_correct == correct and print_fails:
            append_list(fail_list, results_5, command_freq, graph_depth)  
                              
    return 'Correct Proportion: {:.2f}% |\n Correct in 5 not 3: {:.2f}% |\n Correct in 15 not 5: {:.2f}% |\n Has Prediction and is Correct: {:.2f}% |\n Incorrect Proportion: {:.2f}% |\n None Proportion: {:.2f}% |\n First Prediction: {:.2f}% |\n Average Success Freq: {:.2f} |\n Average Fail Freq: {:.2f}'.format(100 * correct/(test_size-none_count), 100 * correct_with_5/(test_size-none_count), 100 * correct_with_15/(test_size-none_count), 100 * correct/has_prediction, 100 * incorrect/(test_size-none_count), 100 * none_count/test_size, 100 * first_prediction/test_size, mean(succeed_frequencies)/len(train_data), mean(fail_frequencies)/len(train_data))


Init parser

In [10]:
from parse import Parser

parser = Parser()

100%|██████████| 56/56 [00:06<00:00,  8.73it/s]


All Commands Parse and Replace Args

In [11]:
subset_size = 5

science_session = parser.parse_commands_per_session(parser.scientists_files)
science_session = parser.parse_commands_into_subsets_sliding_window(science_session, subset_size)
science_session = parser.replace_args_nested(science_session)

experienced_session = parser.parse_commands_per_session(parser.experienced_files)
experienced_session = parser.parse_commands_into_subsets_sliding_window(experienced_session, subset_size)
experienced_session = parser.replace_args_nested(experienced_session)

non_session = parser.parse_commands_per_session(parser.non_programmers_files)
non_session = parser.parse_commands_into_subsets_sliding_window(non_session, subset_size)
non_session = parser.replace_args_nested(non_session)

novice_session = parser.parse_commands_per_session(parser.novice_files)
novice_session = parser.parse_commands_into_subsets_sliding_window(novice_session, subset_size)
novice_session = parser.replace_args_nested(novice_session)

100%|██████████| 56/56 [00:06<00:00,  8.30it/s]


Predicting last command from 1..n-1 commands

In [6]:
i = -1

print("Scientist session:\n", get_accuracy(science_session, graph_depth=i), '\n')
print("Experienced session:\n", get_accuracy(experienced_session, graph_depth=i), '\n')
print("Non programmer session:\n", get_accuracy(non_session, graph_depth=i), '\n')
print("Novice session:\n", get_accuracy(novice_session, graph_depth=i), '\n')

Scientist session:
 Correct Proportion: 41.87% |
 Correct in 5 not 3: 48.35% |
 Correct in 15 not 5: 60.57% |
 Has Prediction and is Correct: 41.87% |
 Incorrect Proportion: 56.23% |
 None Proportion: 3.27% |
 First Prediction: 26.27% |
 Average Success Freq: 0.02 |
 Average Fail Freq: 0.02 



Experienced session:
 Correct Proportion: 44.55% |
 Correct in 5 not 3: 51.53% |
 Correct in 15 not 5: 62.86% |
 Has Prediction and is Correct: 44.55% |
 Incorrect Proportion: 53.61% |
 None Proportion: 3.31% |
 First Prediction: 27.23% |
 Average Success Freq: 0.04 |
 Average Fail Freq: 0.04 



Non programmer session:
 Correct Proportion: 45.25% |
 Correct in 5 not 3: 53.45% |
 Correct in 15 not 5: 65.63% |
 Has Prediction and is Correct: 45.25% |
 Incorrect Proportion: 53.73% |
 None Proportion: 1.87% |
 First Prediction: 27.29% |
 Average Success Freq: 0.14 |
 Average Fail Freq: 0.14 



Novice session:
 Correct Proportion: 64.12% |
 Correct in 5 not 3: 70.11% |
 Correct in 15 not 5: 78.49% |
 Has Prediction and is Correct: 64.12% |
 Incorrect Proportion: 35.33% |
 None Proportion: 1.51% |
 First Prediction: 40.91% |
 Average Success Freq: 0.06 |
 Average Fail Freq: 0.05 



Predicting N from 1..N-1 for all

In [7]:
i = -1

all = science_session + experienced_session + non_session + novice_session

print(get_accuracy(all, graph_depth=i))

Correct Proportion: 59.05% |
 Correct in 5 not 3: 66.01% |
 Correct in 15 not 5: 77.76% |
 Has Prediction and is Correct: 59.05% |
 Incorrect Proportion: 40.78% |
 None Proportion: 0.43% |
 First Prediction: 39.82% |
 Average Success Freq: 0.03 |
 Average Fail Freq: 0.03


Predicting for all

In [8]:
all = science_session + experienced_session + non_session + novice_session

for i in range(1, 5):
    print("Accuracy predicting command {} from commands {} through {}:\n".format(i+1, 1, i))

    print(get_accuracy(all, graph_depth=i), '\n')

Accuracy predicting command 2 from commands 1 through 1:



Correct Proportion: 42.63% |
 Correct in 5 not 3: 50.50% |
 Correct in 15 not 5: 66.32% |
 Has Prediction and is Correct: 42.63% |
 Incorrect Proportion: 57.37% |
 None Proportion: 0.00% |
 First Prediction: 25.77% |
 Average Success Freq: 0.05 |
 Average Fail Freq: 0.05 

Accuracy predicting command 3 from commands 1 through 2:



Correct Proportion: 56.81% |
 Correct in 5 not 3: 64.96% |
 Correct in 15 not 5: 79.43% |
 Has Prediction and is Correct: 56.81% |
 Incorrect Proportion: 43.19% |
 None Proportion: 0.00% |
 First Prediction: 36.53% |
 Average Success Freq: 0.05 |
 Average Fail Freq: 0.05 

Accuracy predicting command 4 from commands 1 through 3:



Correct Proportion: 68.78% |
 Correct in 5 not 3: 76.36% |
 Correct in 15 not 5: 87.73% |
 Has Prediction and is Correct: 68.78% |
 Incorrect Proportion: 31.21% |
 None Proportion: 0.03% |
 First Prediction: 44.13% |
 Average Success Freq: 0.05 |
 Average Fail Freq: 0.05 

Accuracy predicting command 5 from commands 1 through 4:



Correct Proportion: 70.00% |
 Correct in 5 not 3: 74.27% |
 Correct in 15 not 5: 79.57% |
 Has Prediction and is Correct: 70.00% |
 Incorrect Proportion: 29.45% |
 None Proportion: 1.81% |
 First Prediction: 53.59% |
 Average Success Freq: 0.06 |
 Average Fail Freq: 0.06 



~500 command sets

In [11]:
from random import sample 
import math

all = science_session + experienced_session + non_session + novice_session
print(len(all))

all = sample(all, math.floor(.0005 * len(all)))
print(len(all))

for i in range(1, 5):
    print("Accuracy predicting command {} from commands {} through {}:\n".format(i+1, 1, i))

    print(get_accuracy(all, graph_depth=i), '\n')

print(get_accuracy(all, graph_depth=-1), '\n')

978861
489
Accuracy predicting command 2 from commands 1 through 1:



Correct Proportion: 53.06% |
 Correct in 5 not 3: 56.12% |
 Correct in 15 not 5: 78.57% |
 Has Prediction and is Correct: 53.06% |
 Incorrect Proportion: 46.94% |
 None Proportion: 0.00% |
 First Prediction: 36.73% |
 Average Success Freq: 141.21 |
 Average Fail Freq: 145.96 

Accuracy predicting command 3 from commands 1 through 2:



Correct Proportion: 60.00% |
 Correct in 5 not 3: 65.71% |
 Correct in 15 not 5: 81.43% |
 Has Prediction and is Correct: 60.00% |
 Incorrect Proportion: 40.00% |
 None Proportion: 0.00% |
 First Prediction: 42.86% |
 Average Success Freq: 141.22 |
 Average Fail Freq: 145.97 

Accuracy predicting command 4 from commands 1 through 3:



Correct Proportion: 75.00% |
 Correct in 5 not 3: 81.25% |
 Correct in 15 not 5: 91.67% |
 Has Prediction and is Correct: 75.00% |
 Incorrect Proportion: 25.00% |
 None Proportion: 0.00% |
 First Prediction: 50.00% |
 Average Success Freq: 141.22 |
 Average Fail Freq: 145.97 

Accuracy predicting command 5 from commands 1 through 4:



Correct Proportion: 86.36% |
 Correct in 5 not 3: 86.36% |
 Correct in 15 not 5: 95.45% |
 Has Prediction and is Correct: 86.36% |
 Incorrect Proportion: 13.64% |
 None Proportion: 0.00% |
 First Prediction: 72.73% |
 Average Success Freq: 141.22 |
 Average Fail Freq: 145.97 



Correct Proportion: 67.35% |
 Correct in 5 not 3: 70.41% |
 Correct in 15 not 5: 84.69% |
 Has Prediction and is Correct: 67.35% |
 Incorrect Proportion: 32.65% |
 None Proportion: 0.00% |
 First Prediction: 50.00% |
 Average Success Freq: 141.24 |
 Average Fail Freq: 145.98 



Around 15000 command subsets

In [20]:
from random import sample
import math

sample_size = .1

science_sample = sample(science_session, math.floor(sample_size * len(science_session)))
experienced_sample = sample(experienced_session, math.floor(sample_size * len(experienced_session)))
non_sample = sample(non_session, math.floor(sample_size * len(non_session)))
novice_sample = sample(novice_session, math.floor(sample_size * len(novice_session)))

all = science_sample + experienced_sample + non_sample + novice_sample
all_sample = sample(all, math.floor(sample_size * len(all)))

for i in range(1, 5):
    print("Accuracy predicting command {} from commands {} through {}:".format(i+1, 1, i))

    print("Scientist session:\n", get_accuracy(science_sample, graph_depth=i))
    print("Experienced session:\n", get_accuracy(experienced_sample, graph_depth=i))
    print("Non programmer session:\n", get_accuracy(non_sample, graph_depth=i))
    print("Novice session:\n", get_accuracy(novice_sample, graph_depth=i))
    print("All commands:\n", get_accuracy(all_sample, graph_depth=i))

i = -1

print("Accuracy predicting command N from commands 1 through N-1:")

print("Scientist session:\n", get_accuracy(science_sample, graph_depth=i))
print("Experienced session:\n", get_accuracy(experienced_sample, graph_depth=i))
print("Non programmer session:\n", get_accuracy(non_sample, graph_depth=i))
print("Novice session:\n", get_accuracy(novice_sample, graph_depth=i))
print("All commands:\n", get_accuracy(all_sample, graph_depth=i))

Accuracy predicting command 2 from commands 1 through 1:


Scientist session:
 Correct Proportion: 33.64% |
 Correct in 5 not 3: 41.20% |
 Correct in 15 not 5: 58.49% |
 Has Prediction and is Correct: 33.64% |
 Incorrect Proportion: 66.14% |
 None Proportion: 0.33% |
 First Prediction: 19.48% |
 Average Success Freq: 0.20 |
 Average Fail Freq: 0.18


Experienced session:
 Correct Proportion: 37.54% |
 Correct in 5 not 3: 46.13% |
 Correct in 15 not 5: 62.18% |
 Has Prediction and is Correct: 37.54% |
 Incorrect Proportion: 62.26% |
 None Proportion: 0.33% |
 First Prediction: 20.34% |
 Average Success Freq: 0.35 |
 Average Fail Freq: 0.32


Non programmer session:
 Correct Proportion: 41.00% |
 Correct in 5 not 3: 47.60% |
 Correct in 15 not 5: 63.93% |
 Has Prediction and is Correct: 41.00% |
 Incorrect Proportion: 58.85% |
 None Proportion: 0.26% |
 First Prediction: 22.43% |
 Average Success Freq: 1.12 |
 Average Fail Freq: 1.04


Novice session:
 Correct Proportion: 59.63% |
 Correct in 5 not 3: 67.78% |
 Correct in 15 not 5: 79.82% |
 Has Prediction and is Correct: 59.63% |
 Incorrect Proportion: 40.23% |
 None Proportion: 0.36% |
 First Prediction: 39.91% |
 Average Success Freq: 0.40 |
 Average Fail Freq: 0.35


...

Scientist session:
 Correct Proportion: 43.73% |
 Correct in 5 not 3: 49.59% |
 Correct in 15 not 5: 61.20% |
 Has Prediction and is Correct: 43.73% |
 Incorrect Proportion: 50.10% |
 None Proportion: 10.95% |
 First Prediction: 26.18% |
 Average Success Freq: 0.29 |
 Average Fail Freq: 0.26


Experienced session:
 Correct Proportion: 47.99% |
 Correct in 5 not 3: 54.70% |
 Correct in 15 not 5: 64.64% |
 Has Prediction and is Correct: 47.99% |
 Incorrect Proportion: 46.53% |
 None Proportion: 10.53% |
 First Prediction: 28.36% |
 Average Success Freq: 0.50 |
 Average Fail Freq: 0.45


Non programmer session:
 Correct Proportion: 48.32% |
 Correct in 5 not 3: 55.38% |
 Correct in 15 not 5: 66.83% |
 Has Prediction and is Correct: 48.32% |
 Incorrect Proportion: 48.18% |
 None Proportion: 6.77% |
 First Prediction: 28.18% |
 Average Success Freq: 1.56 |
 Average Fail Freq: 1.42


Novice session:
 Correct Proportion: 65.33% |
 Correct in 5 not 3: 70.84% |
 Correct in 15 not 5: 79.60% |
 Has Prediction and is Correct: 65.33% |
 Incorrect Proportion: 32.80% |
 None Proportion: 5.38% |
 First Prediction: 40.31% |
 Average Success Freq: 0.52 |
 Average Fail Freq: 0.46


All commands:
 Correct Proportion: 64.18% |
 Correct in 5 not 3: 70.69% |
 Correct in 15 not 5: 81.72% |
 Has Prediction and is Correct: 64.18% |
 Incorrect Proportion: 34.83% |
 None Proportion: 2.76% |
 First Prediction: 45.61% |
 Average Success Freq: 1.34 |
 Average Fail Freq: 1.18


Accuracy predicting N+1 command from N commands with all commands in graph

In [13]:
for i in range(1, 5):
    print("Accuracy predicting command {} from commands {} through {}:\n".format(i+1, 1, i))

    print("Scientist session:\n", get_accuracy(science_session, graph_depth=i), '\n')
    print("Experienced session:\n", get_accuracy(experienced_session, graph_depth=i), '\n')
    print("Non programmer session:\n", get_accuracy(non_session, graph_depth=i), '\n')
    print("Novice session:\n", get_accuracy(novice_session, graph_depth=i), '\n')

Accuracy predicting command 2 from commands 1 through 1:



Scientist session:
 Correct Proportion: 35.74% |
 Correct in 5 not 3: 42.81% |
 Correct in 15 not 5: 60.13% |
 Has Prediction and is Correct: 35.74% |
 Incorrect Proportion: 64.26% |
 None Proportion: 0.00% |
 First Prediction: 20.47% |
 Average Success Freq: 0.19 |
 Average Fail Freq: 0.21 



Experienced session:
 Correct Proportion: 37.64% |
 Correct in 5 not 3: 46.63% |
 Correct in 15 not 5: 64.04% |
 Has Prediction and is Correct: 37.64% |
 Incorrect Proportion: 62.36% |
 None Proportion: 0.00% |
 First Prediction: 20.66% |
 Average Success Freq: 0.33 |
 Average Fail Freq: 0.37 



Non programmer session:
 Correct Proportion: 39.30% |
 Correct in 5 not 3: 46.44% |
 Correct in 15 not 5: 65.05% |
 Has Prediction and is Correct: 39.30% |
 Incorrect Proportion: 60.70% |
 None Proportion: 0.00% |
 First Prediction: 22.07% |
 Average Success Freq: 1.04 |
 Average Fail Freq: 1.19 



Novice session:
 Correct Proportion: 60.57% |
 Correct in 5 not 3: 68.89% |
 Correct in 15 not 5: 80.18% |
 Has Prediction and is Correct: 60.57% |
 Incorrect Proportion: 39.43% |
 None Proportion: 0.00% |
 First Prediction: 40.90% |
 Average Success Freq: 0.37 |
 Average Fail Freq: 0.39 

Accuracy predicting command 3 from commands 1 through 2:



...

Novice session:
 Correct Proportion: 78.04% |
 Correct in 5 not 3: 84.18% |
 Correct in 15 not 5: 92.69% |
 Has Prediction and is Correct: 78.04% |
 Incorrect Proportion: 21.95% |
 None Proportion: 0.02% |
 First Prediction: 41.89% |
 Average Success Freq: 0.41 |
 Average Fail Freq: 0.44 

Accuracy predicting command 5 from commands 1 through 4:



Scientist session:
 Correct Proportion: 70.55% |
 Correct in 5 not 3: 74.53% |
 Correct in 15 not 5: 80.12% |
 Has Prediction and is Correct: 70.55% |
 Incorrect Proportion: 28.85% |
 None Proportion: 2.03% |
 First Prediction: 54.76% |
 Average Success Freq: 0.25 |
 Average Fail Freq: 0.27 



Experienced session:
 Correct Proportion: 73.89% |
 Correct in 5 not 3: 78.08% |
 Correct in 15 not 5: 82.62% |
 Has Prediction and is Correct: 73.89% |
 Incorrect Proportion: 25.57% |
 None Proportion: 2.09% |
 First Prediction: 57.73% |
 Average Success Freq: 0.42 |
 Average Fail Freq: 0.45 



Non programmer session:
 Correct Proportion: 68.87% |
 Correct in 5 not 3: 75.08% |
 Correct in 15 not 5: 82.73% |
 Has Prediction and is Correct: 68.87% |
 Incorrect Proportion: 30.76% |
 None Proportion: 1.17% |
 First Prediction: 48.29% |
 Average Success Freq: 1.32 |
 Average Fail Freq: 1.40 



Novice session:
 Correct Proportion: 79.67% |
 Correct in 5 not 3: 83.60% |
 Correct in 15 not 5: 88.01% |
 Has Prediction and is Correct: 79.67% |
 Incorrect Proportion: 20.20% |
 None Proportion: 0.64% |
 First Prediction: 61.12% |
 Average Success Freq: 0.42 |
 Average Fail Freq: 0.44 



Consolidated predicting next command and saves it. Graph depth of `-1` should accomplish this.

In [20]:
import os
import csv

dataset_names = ["Science data", "Experienced data", "Non programmer data", "Novice data"]
datasets = [science_session, experienced_session, non_session, novice_session]

for i in range(len(datasets)):
    fail_frequencies = []
    succeed_frequencies = []

    print(get_accuracy(datasets[i], fail_frequencies=fail_frequencies, succeed_frequencies=succeed_frequencies, print_fails=True, graph_depth=-1))

    os.makedirs(os.path.dirname("fails/frequencies_all/" + dataset_names[i] + ".csv"), exist_ok=True)
    with open("fails/frequencies_all/" + dataset_names[i] + ".csv", "w", newline='') as file:
        writer = csv.writer(file)
        writer.writerow(fail_frequencies)

    os.makedirs(os.path.dirname("successes/frequencies_all/" + dataset_names[i] + ".csv"), exist_ok=True)
    with open("successes/frequencies_all/" + dataset_names[i] + ".csv", "w", newline='') as file:
        writer = csv.writer(file)
        writer.writerow(succeed_frequencies)


Correct Proportion: 41.87% |
 Correct in 5 not 3: 48.35% |
 Correct in 15 not 5: 60.57% |
 Has Prediction and is Correct: 41.87% |
 Incorrect Proportion: 56.23% |
 None Proportion: 3.27% |
 First Prediction: 26.27% |
 Average Success Freq: 0.02 |
 Average Fail Freq: 0.02


Correct Proportion: 44.55% |
 Correct in 5 not 3: 51.53% |
 Correct in 15 not 5: 62.86% |
 Has Prediction and is Correct: 44.55% |
 Incorrect Proportion: 53.61% |
 None Proportion: 3.31% |
 First Prediction: 27.23% |
 Average Success Freq: 0.05 |
 Average Fail Freq: 0.05


Correct Proportion: 45.25% |
 Correct in 5 not 3: 53.45% |
 Correct in 15 not 5: 65.63% |
 Has Prediction and is Correct: 45.25% |
 Incorrect Proportion: 53.73% |
 None Proportion: 1.87% |
 First Prediction: 27.29% |
 Average Success Freq: 0.20 |
 Average Fail Freq: 0.22


Correct Proportion: 64.12% |
 Correct in 5 not 3: 70.11% |
 Correct in 15 not 5: 78.49% |
 Has Prediction and is Correct: 64.12% |
 Incorrect Proportion: 35.33% |
 None Proportion: 1.51% |
 First Prediction: 40.91% |
 Average Success Freq: 0.10 |
 Average Fail Freq: 0.08


Save files for all data

In [22]:
import os
import pprint

pp = pprint.PrettyPrinter(indent=4)

dataset_names = ["Science data", "Experienced data", "Non programmer data", "Novice data"]
datasets = [science_session, experienced_session, non_session, novice_session]

for i in range(len(datasets)):
    for j in range (1, 5):
        fail_list = []
        succeed_list = []
        correct_5_not_3 = []
        correct_15_not_5 = []

        get_accuracy(datasets[i], fail_list=fail_list, succeed_list=succeed_list, correct_5_not_3=correct_5_not_3, correct_15_not_5=correct_15_not_5, print_fails=True, graph_depth=j)

        os.makedirs(os.path.dirname("fails/predicting_" + str(j+1) + "/" + dataset_names[i] + ".txt"), exist_ok=True)
        with open("fails/predicting_" + str(j+1) + "/" + dataset_names[i] + ".txt", "w") as file:
            file.write(pp.pformat(fail_list))

        os.makedirs(os.path.dirname("successes/predicting_" + str(j+1) + "/" + dataset_names[i] + ".txt"), exist_ok=True)
        with open("successes/predicting_" + str(j+1) + "/" + dataset_names[i] + ".txt", "w") as file:
            file.write(pp.pformat(succeed_list))

        os.makedirs(os.path.dirname("correct_5_not_3/predicting_" + str(j+1) + "/" + dataset_names[i] + ".txt"), exist_ok=True)
        with open("correct_5_not_3/predicting_" + str(j+1) + "/" + dataset_names[i] + ".txt", "w") as file:
            file.write(pp.pformat(correct_5_not_3))

        os.makedirs(os.path.dirname("correct_15_not_5/predicting_" + str(j+1) + "/" + dataset_names[i] + ".txt"), exist_ok=True)
        with open("correct_15_not_5/predicting_" + str(j+1) + "/" + dataset_names[i] + ".txt", "w") as file:
            file.write(pp.pformat(correct_15_not_5))


Save frequencies for successes and failures to CSV

In [23]:
import os
import csv

dataset_names = ["Science data", "Experienced data", "Non programmer data", "Novice data"]
datasets = [science_session, experienced_session, non_session, novice_session]

for i in range(len(datasets)):
    for j in range (1, 5):
        fail_frequencies = []
        succeed_frequencies = []

        get_accuracy(datasets[i], fail_frequencies=fail_frequencies, succeed_frequencies=succeed_frequencies, print_fails=True, graph_depth=j)

        os.makedirs(os.path.dirname("fails/frequencies_" + str(j+1) + "/" + dataset_names[i] + ".csv"), exist_ok=True)
        with open("fails/frequencies_" + str(j+1) + "/" + dataset_names[i] + ".csv", "w", newline='') as file:
            writer = csv.writer(file)
            writer.writerow(fail_frequencies)

        os.makedirs(os.path.dirname("successes/frequencies_" + str(j+1) + "/" + dataset_names[i] + ".csv"), exist_ok=True)
        with open("successes/frequencies_" + str(j+1) + "/" + dataset_names[i] + ".csv", "w", newline='') as file:
            writer = csv.writer(file)
            writer.writerow(succeed_frequencies)


Sampled Commands Parse and Replace Args

Change to sample at end stage after replacing args not for parsing by session

In [34]:
from random import sample
import math

subset_size = 5
sample_size = .2

science_session_sample = parser.parse_commands_per_session(sample(parser.scientists_files, math.floor(sample_size * len(parser.scientists_files))))
science_session_sample = parser.parse_commands_into_subsets_sliding_window(science_session_sample, subset_size)
science_session_sample = parser.replace_args_nested(science_session_sample)

experienced_session_sample = parser.parse_commands_per_session(sample(parser.experienced_files, math.floor(sample_size * len(parser.experienced_files))))
experienced_session_sample = parser.parse_commands_into_subsets_sliding_window(experienced_session_sample, subset_size)
experienced_session_sample = parser.replace_args_nested(experienced_session_sample)

non_session_sample = parser.parse_commands_per_session(sample(parser.non_programmers_files, math.floor(sample_size * len(parser.non_programmers_files))))
non_session_sample = parser.parse_commands_into_subsets_sliding_window(non_session_sample, subset_size)
non_session_sample = parser.replace_args_nested(non_session_sample)

novice_session_sample = parser.parse_commands_per_session(sample(parser.novice_files, math.floor(sample_size * len(parser.novice_files))))
novice_session_sample = parser.parse_commands_into_subsets_sliding_window(novice_session_sample, subset_size)
novice_session_sample = parser.replace_args_nested(novice_session_sample)

100%|██████████| 11/11 [00:02<00:00,  4.74it/s]


Accuracy predicting N+1 command from N commands with subset of data

In [35]:
all = science_session_sample + experienced_session_sample + non_session_sample + novice_session_sample
print(len(all))

for i in range(1, 5):
    print("Accuracy Predicting Command {} from commands {} through {}:\n".format(i+1, 1, i))

    print("Scientist session sample:\n", get_accuracy(science_session_sample, graph_depth=i))
    print("Experienced session sample:\n", get_accuracy(experienced_session_sample, graph_depth=i))
    print("Non programmer session sample:\n", get_accuracy(non_session_sample, graph_depth=i))
    print("Novice session sample:\n", get_accuracy(novice_session_sample, graph_depth=i))
    print("All sample session:\n", get_accuracy(all, graph_depth=i))

print("Accuracy Predicting Command N from commands 1 through N-1")

print("Scientist session sample:\n", get_accuracy(science_session_sample, graph_depth=-1))
print("Experienced session sample:\n", get_accuracy(experienced_session_sample, graph_depth=-1))
print("Non programmer session sample:\n", get_accuracy(non_session_sample, graph_depth=-1))
print("Novice session sample:\n", get_accuracy(novice_session_sample, graph_depth=-1))
print("All sample session:\n", get_accuracy(all, graph_depth=-1))

187420
Accuracy Predicting Command 2 from commands 1 through 1:



Scientist session sample:
 Correct Proportion: 41.53% |
 Correct in 5 not 3: 50.18% |
 Correct in 15 not 5: 69.45% |
 Has Prediction and is Correct: 41.53% |
 Incorrect Proportion: 58.40% |
 None Proportion: 0.10% |
 First Prediction: 23.73% |
 Average Success Freq: 0.03 |
 Average Fail Freq: 0.03


Experienced session sample:
 Correct Proportion: 41.54% |
 Correct in 5 not 3: 50.29% |
 Correct in 15 not 5: 68.92% |
 Has Prediction and is Correct: 41.54% |
 Incorrect Proportion: 58.41% |
 None Proportion: 0.07% |
 First Prediction: 23.38% |
 Average Success Freq: 0.08 |
 Average Fail Freq: 0.08


Non programmer session sample:
 Correct Proportion: 42.86% |
 Correct in 5 not 3: 49.76% |
 Correct in 15 not 5: 70.80% |
 Has Prediction and is Correct: 42.86% |
 Incorrect Proportion: 57.02% |
 None Proportion: 0.21% |
 First Prediction: 25.30% |
 Average Success Freq: 0.33 |
 Average Fail Freq: 0.33


Novice session sample:
 Correct Proportion: 60.51% |
 Correct in 5 not 3: 67.54% |
 Correct in 15 not 5: 80.05% |
 Has Prediction and is Correct: 60.51% |
 Incorrect Proportion: 39.46% |
 None Proportion: 0.08% |
 First Prediction: 39.73% |
 Average Success Freq: 0.09 |
 Average Fail Freq: 0.07


...

Scientist session sample:
 Correct Proportion: 60.41% |
 Correct in 5 not 3: 67.56% |
 Correct in 15 not 5: 79.16% |
 Has Prediction and is Correct: 60.41% |
 Incorrect Proportion: 39.35% |
 None Proportion: 0.60% |
 First Prediction: 41.68% |
 Average Success Freq: 0.14 |
 Average Fail Freq: 0.15


Experienced session sample:
 Correct Proportion: 62.49% |
 Correct in 5 not 3: 69.19% |
 Correct in 15 not 5: 80.29% |
 Has Prediction and is Correct: 62.49% |
 Incorrect Proportion: 37.16% |
 None Proportion: 0.92% |
 First Prediction: 43.90% |
 Average Success Freq: 0.32 |
 Average Fail Freq: 0.33


Non programmer session sample:
 Correct Proportion: 56.52% |
 Correct in 5 not 3: 63.15% |
 Correct in 15 not 5: 76.94% |
 Has Prediction and is Correct: 56.52% |
 Incorrect Proportion: 43.14% |
 None Proportion: 0.79% |
 First Prediction: 35.67% |
 Average Success Freq: 1.36 |
 Average Fail Freq: 1.45


Novice session sample:
 Correct Proportion: 71.23% |
 Correct in 5 not 3: 77.16% |
 Correct in 15 not 5: 87.22% |
 Has Prediction and is Correct: 71.23% |
 Incorrect Proportion: 28.66% |
 None Proportion: 0.37% |
 First Prediction: 47.93% |
 Average Success Freq: 0.28 |
 Average Fail Freq: 0.28


All sample session:
 Correct Proportion: 63.81% |
 Correct in 5 not 3: 70.14% |
 Correct in 15 not 5: 81.65% |
 Has Prediction and is Correct: 63.81% |
 Incorrect Proportion: 35.95% |
 None Proportion: 0.66% |
 First Prediction: 43.86% |
 Average Success Freq: 0.09 |
 Average Fail Freq: 0.10


Predict N from 1..n-1 in samples

In [36]:
i=-1

print("Scientist session sample:\n", get_accuracy(science_session_sample, graph_depth=i))
print("Experienced session sample:\n", get_accuracy(experienced_session_sample, graph_depth=i))
print("Non programmer session sample:\n", get_accuracy(non_session_sample, graph_depth=i))
print("Novice session sample:\n", get_accuracy(novice_session_sample, graph_depth=i))

Scientist session sample:
 Correct Proportion: 60.42% |
 Correct in 5 not 3: 67.55% |
 Correct in 15 not 5: 79.16% |
 Has Prediction and is Correct: 60.42% |
 Incorrect Proportion: 39.34% |
 None Proportion: 0.60% |
 First Prediction: 41.67% |
 Average Success Freq: 0.21 |
 Average Fail Freq: 0.23


Experienced session sample:
 Correct Proportion: 62.49% |
 Correct in 5 not 3: 69.19% |
 Correct in 15 not 5: 80.29% |
 Has Prediction and is Correct: 62.49% |
 Incorrect Proportion: 37.16% |
 None Proportion: 0.92% |
 First Prediction: 43.90% |
 Average Success Freq: 0.45 |
 Average Fail Freq: 0.50


Non programmer session sample:
 Correct Proportion: 56.52% |
 Correct in 5 not 3: 63.15% |
 Correct in 15 not 5: 76.94% |
 Has Prediction and is Correct: 56.52% |
 Incorrect Proportion: 43.14% |
 None Proportion: 0.79% |
 First Prediction: 35.67% |
 Average Success Freq: 1.93 |
 Average Fail Freq: 2.14


Novice session sample:
 Correct Proportion: 71.23% |
 Correct in 5 not 3: 77.16% |
 Correct in 15 not 5: 87.22% |
 Has Prediction and is Correct: 71.23% |
 Incorrect Proportion: 28.66% |
 None Proportion: 0.37% |
 First Prediction: 47.93% |
 Average Success Freq: 0.38 |
 Average Fail Freq: 0.41


Expand pipes and replace args

In [37]:
science = parser.filter_commands_with_pipe(parser.scientists_commands)
experienced = parser.filter_commands_with_pipe(parser.experienced_commands)
non = parser.filter_commands_with_pipe(parser.non_programmers_commands)
novice = parser.filter_commands_with_pipe(parser.novice_commands)

science_pipes = parser.expand_piped_commands(science)
experienced_pipes = parser.expand_piped_commands(experienced)
non_pipes = parser.expand_piped_commands(non)
novice_pipes = parser.expand_piped_commands(novice)

science_pipes_expand = parser.replace_arg_expanded_pipe(science_pipes)
experienced_pipes_expand = parser.replace_arg_expanded_pipe(experienced_pipes)
non_pipes_expand = parser.replace_arg_expanded_pipe(non_pipes)
novice_pipes_expand = parser.replace_arg_expanded_pipe(novice_pipes)

In [38]:
print("Science pipes metrics:\n", get_accuracy(science_pipes_expand), "\n")
print("Experienced pipes metrics:\n", get_accuracy(experienced_pipes_expand), "\n")
print("Non programmers pipes metrics:\n", get_accuracy(non_pipes_expand), "\n")
print("Novice pipes metrics:\n", get_accuracy(novice_pipes_expand), "\n")

[]


Science pipes metrics:
 Correct Proportion: 40.17% |
 Correct in 5 not 3: 41.47% |
 Correct in 15 not 5: 51.62% |
 Has Prediction and is Correct: 40.17% |
 Incorrect Proportion: 58.32% |
 None Proportion: 2.53% |
 First Prediction: 30.74% |
 Average Success Freq: 8.17 |
 Average Fail Freq: 8.79 



Experienced pipes metrics:
 Correct Proportion: 23.26% |
 Correct in 5 not 3: 26.05% |
 Correct in 15 not 5: 53.49% |
 Has Prediction and is Correct: 23.26% |
 Incorrect Proportion: 75.34% |
 None Proportion: 1.83% |
 First Prediction: 18.26% |
 Average Success Freq: 17.81 |
 Average Fail Freq: 19.16 



Non programmers pipes metrics:
 Correct Proportion: 24.17% |
 Correct in 5 not 3: 25.54% |
 Correct in 15 not 5: 80.70% |
 Has Prediction and is Correct: 24.17% |
 Incorrect Proportion: 75.83% |
 None Proportion: 0.00% |
 First Prediction: 21.25% |
 Average Success Freq: 7.54 |
 Average Fail Freq: 8.10 



Novice pipes metrics:
 Correct Proportion: 25.58% |
 Correct in 5 not 3: 25.58% |
 Correct in 15 not 5: 25.58% |
 Has Prediction and is Correct: 25.58% |
 Incorrect Proportion: 72.73% |
 None Proportion: 2.27% |
 First Prediction: 0.00% |
 Average Success Freq: 89.91 |
 Average Fail Freq: 96.66 

